<a href="https://colab.research.google.com/github/Abdalla-Ghanem-MCSD/Abdalla-Ghanem/blob/master/Abdalla_Tabnet_Sqlinjection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-tabnet scikit-learn pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sql_injection_dataset.csv to sql_injection_dataset.csv


In [ ]:
import pandas as pd


df = pd.read_csv("Modified_SQL_Dataset.csv")
df.head()

,Query,Label
0,""" or pg_sleep ( __TIME__ ) --",1
1,create user name identified by pass123 tempora...,1
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Convert text into numeric form (TF-IDF)
vectorizer = TfidfVectorizer(max_features=100)
X = vectorizer.fit_transform(df["Query"]).toarray()
y = df["Label"].astype(int).values

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
!pip install pytorch-tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

clf = TabNetClassifier()

clf.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric=['accuracy'],
    max_epochs=100,
    patience=10,
    batch_size=1024
)


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.66684 | val_0_accuracy: 0.62953 |  0:00:02s
epoch 1  | loss: 0.43052 | val_0_accuracy: 0.62613 |  0:00:05s
epoch 2  | loss: 0.25518 | val_0_accuracy: 0.6216  |  0:00:07s
epoch 3  | loss: 0.13576 | val_0_accuracy: 0.62419 |  0:00:09s
epoch 4  | loss: 0.10224 | val_0_accuracy: 0.68418 |  0:00:12s
epoch 5  | loss: 0.08206 | val_0_accuracy: 0.70246 |  0:00:14s
epoch 6  | loss: 0.07773 | val_0_accuracy: 0.75356 |  0:00:17s
epoch 7  | loss: 0.0744  | val_0_accuracy: 0.78703 |  0:00:20s
epoch 8  | loss: 0.07011 | val_0_accuracy: 0.76375 |  0:00:22s
epoch 9  | loss: 0.0708  | val_0_accuracy: 0.75776 |  0:00:25s
epoch 10 | loss: 0.06854 | val_0_accuracy: 0.86158 |  0:00:27s
epoch 11 | loss: 0.06533 | val_0_accuracy: 0.90411 |  0:00:29s
epoch 12 | loss: 0.064   | val_0_accuracy: 0.89554 |  0:00:31s
epoch 13 | loss: 0.0623  | val_0_accuracy: 0.93871 |  0:00:34s
epoch 14 | loss: 0.06091 | val_0_accuracy: 0.94211 |  0:00:36s
epoch 15 | loss: 0.06825 | val_0_accuracy: 0.91882 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = clf.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("📊 Classification Report:\n", classification_report(y_test, y_pred))



✅ Accuracy: 0.9864165588615783
📊 Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      3893
           1       0.99      0.97      0.98      2291

    accuracy                           0.99      6184
   macro avg       0.99      0.98      0.99      6184
weighted avg       0.99      0.99      0.99      6184



In [ ]:
def detect_sql_injection(user_input: str):
    input_vector = vectorizer.transform([user_input]).toarray()
    prediction = clf.predict(input_vector)
    return "⚠️ SQL Injection Detected" if prediction[0] == 1 else "✅ Normal Input"

# Test with a real query
query = input("🔐 Enter SQL query or username: ")
print("🔎 Detection Result:", detect_sql_injection(query))


🔐 Enter SQL query or username:  select * from users where id  =  1 or "%[" or 1  =  1 -- 1
🔎 Detection Result: ⚠️ SQL Injection Detected
